# OvertureMaestro Command Line Interface

**OvertureMaestro** contains a CLI for users convenience. It is **not** installed by default when installed using `pip install overturemaestro`.

To include the CLI, **OvertureMaestro** has to be installed with additional group called `cli`: `pip install overturemaestro[cli]`.

CLI is based on the `typer` library and exposes almost all of the features implemented in the Python API.

After installation, the `OvertureMaestro` (or `overturemaestro`) command will be available in the shell.

Each command error returns a verbose description what went wrong.

In [ ]:
# Extend the default console width from 80 characters
import os

os.environ["COLUMNS"] = "160"

## Basic usage

By default, the overturemaestro requires just the path to the `PBF` file. Without it, there will be an error.

In [ ]:
! OvertureMaestro

Let's download a small extract of buildings in Monaco

In [ ]:
! OvertureMaestro buildings building --geom-filter-bbox 7.416486,43.730886,7.421931,43.733507

Second execution of this command will immediately return a path to the previously generated file.

In [ ]:
! OvertureMaestro buildings building --geom-filter-bbox 7.416486,43.730886,7.421931,43.733507

To force the regeneration of the GeoParquet file, add the `--ignore-cache` flag (or `--no-cache`) to the command.

In [ ]:
! OvertureMaestro buildings building --geom-filter-bbox 7.416486,43.730886,7.421931,43.733507 --ignore-cache

You can also set the output file path using `-o` (or `--output`) option.

In [ ]:
! OvertureMaestro buildings building --geom-filter-bbox 7.416486,43.730886,7.421931,43.733507 -o monaco_buildings.parquet

You can quickly inspect the data using [`pixel-map`](https://github.com/RaczeQ/pixel-map) tool that displays the geo data in the terminal.

In [ ]:
! pixel-map monaco_buildings.parquet --width 82 --height 23 --renderer ascii-bw

## Help command

To get the full description of all arguments of the OvertureMaestro command, you can use the `--help` (or `-h`) parameter.

In [ ]:
! OvertureMaestro --help

## Geometry filters

`OvertureMaestro` will automatically download required data based on multiple geometry filters:
- Text to geocode using Nominatim
- WKT geometry
- GeoJSON geometry
- Geometry file path
- H3 spatial index
- Geohash spatial index
- S2 spatial index

`OvertureMaestro` will raise an error if provided geometry has parts without area (such as Points, LineStrings or empty geometry).

Let's see the example based on Monaco region.

First, we will visualise multiple filters on the map.

In [ ]:
import geopandas as gpd

from overturemaestro.cli import (
    BboxGeometryParser,
    GeocodeGeometryParser,
    GeohashGeometryParser,
    GeoJsonGeometryParser,
    H3GeometryParser,
    S2GeometryParser,
    WktGeometryParser,
)

In [ ]:
bbox_string = "7.416486,43.730886,7.421931,43.733507"
geocode_string = "Monaco-Ville, Monaco"
geojson_string = """{"type":"Feature","geometry":{"coordinates":[[[7.416,43.734],[7.416,43.731],[7.421,43.731],[7.421,43.734],[7.416,43.734]]],"type":"Polygon"}}"""
wkt_string = "POLYGON ((7.414 43.735, 7.414 43.732, 7.419 43.732, 7.419 43.735, 7.414 43.735))"
h3_string = "893969a4037ffff"
geohash_string = "spv2bcs"
s2_string = "12cdc28dc"

In [ ]:
geometry_types = ["BBox", "Geocode", "GeoJSON", "WKT", "H3", "GeoHash", "S2"]
geometries = [
    BboxGeometryParser().convert(bbox_string),
    GeocodeGeometryParser().convert(geocode_string),
    GeoJsonGeometryParser().convert(geojson_string),
    WktGeometryParser().convert(wkt_string),
    H3GeometryParser().convert(h3_string),
    GeohashGeometryParser().convert(geohash_string),
    S2GeometryParser().convert(s2_string),
]
gpd.GeoDataFrame(
    data=dict(type=geometry_types),
    geometry=geometries,
    crs=4326,
).explore(column="type", tiles="CartoDB positron")

Now we will execute each filter and let OvertureMaestro find required region on its own.

During first execution, OvertureMaestro will cache three PBF files sources locally. This operation takes some time.

The `--silent` flag will disable the progress output to the terminal.

In [ ]:
! OvertureMaestro buildings building --geom-filter-bbox 7.416486,43.730886,7.421931,43.733507 --silent --output files/bbox_example.parquet

### Geocoding

In [ ]:
! OvertureMaestro buildings building --geom-filter-geocode 'Monaco-Ville, Monaco' --silent --output files/geocode_example.parquet

### GeoJSON

In [ ]:
! OvertureMaestro buildings building \
    --geom-filter-geojson '{"type":"Feature","geometry":{"coordinates":[[[7.416,43.734],[7.416,43.731],[7.421,43.731],[7.421,43.734],[7.416,43.734]]],"type":"Polygon"}}' \
    --silent --output files/geojson_example.parquet

### Geohash

In [ ]:
! OvertureMaestro buildings building --geom-filter-index-geohash spv2bcs --silent --output files/geohash_example.parquet

### H3

In [ ]:
! OvertureMaestro buildings building --geom-filter-index-h3 893969a4037ffff --silent --output files/h3_example.parquet

### S2

In [ ]:
! OvertureMaestro buildings building --geom-filter-index-s2 12cdc28dc --silent --output files/s2_example.parquet

### WKT

In [ ]:
! OvertureMaestro buildings building --geom-filter-wkt 'POLYGON ((7.414 43.735, 7.414 43.732, 7.419 43.732, 7.419 43.735, 7.414 43.735))' \
    --silent --output files/wkt_example.parquet

Plot all results for comparison

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 4, sharex=True, sharey=True, figsize=(10, 5))

for idx, (geometry_type, geometry) in enumerate(zip(geometry_types, geometries)):
    ax = axs[idx // 4, idx % 4]
    gdf = gpd.read_parquet(f"files/{geometry_type.lower()}_example.parquet")
    gdf.plot(ax=ax, markersize=1, zorder=1, alpha=0.8)
    gdf.boundary.plot(ax=ax, markersize=0, zorder=1, alpha=0.8)
    gpd.GeoSeries([geometry], crs=4326).plot(
        ax=ax,
        color=(0, 0, 0, 0),
        zorder=2,
        hatch="///",
        edgecolor="orange",
        linewidth=1.5,
    )
    ax.set_title(geometry_type)

axs[1, 3].set_axis_off()

blue_patch = mpatches.Patch(color="C0", alpha=0.8, label="OSM features")
orange_patch = mpatches.Patch(
    facecolor=(0, 0, 0, 0), edgecolor="orange", hatch="///", linewidth=1.5, label="Geometry filter"
)
fig.legend(handles=[blue_patch, orange_patch], bbox_to_anchor=(0.98, 0.35))

fig.tight_layout()

## Theme and type 

**Overture Maps** data uses `themes` and `types` to partition the dataset by function. User can retrieve this list for any available release version supported by `OvertureMaestro`.

Full definition is explained in the [official schema documentation](https://docs.overturemaps.org/schema/concepts/).

### Displaying available extracts

CLI exposes a dedicated flag `--show-extracts` (or `--show-osm-extracts`) for display a list of available extracts.

You can read more about it in a dedicated [OSM extracts example](../advanced_examples/osm_extracts/#display-available-osm-extracts).

Without providing an OSM extract source (`--osm-extract-source`), all sources will be displayed at once.

<style>
div.jp-Cell-outputArea pre {
  overflow-y: auto;
  max-height: 50vh;
}
</style>

In [ ]:
! OvertureMaestro --show-theme-type-pairs

Let's see the example based on the London region.

### Buildings

In [ ]:
! OvertureMaestro buildings building --geom-filter-geocode "City of London" --silent --output files/london_buildings_example.parquet

### Places

In [ ]:
! OvertureMaestro places place --geom-filter-geocode "City of London" --silent --output files/london_places_example.parquet

### Water

In [ ]:
! OvertureMaestro base water --geom-filter-geocode "City of London" --silent --output files/london_water_example.parquet

### Roads

In [ ]:
! OvertureMaestro transportation segment --geom-filter-geocode "City of London" \
    --filter "subtype == road" --silent --output files/london_roads_example.parquet

Plot all different types of features for comparison

In [ ]:
import matplotlib.pyplot as plt

from overturemaestro import geocode_to_geometry

geometry_filter = gpd.GeoSeries([geocode_to_geometry("City of London")], crs=4326)
bounds = geometry_filter.total_bounds
geometry_types = ["water", "roads", "buildings", "places"]
colors = ["#118AB2", "#073B4C", "#06D6A0", "#FFD166"]

fig, axs = plt.subplot_mosaic(
    """
    aa
    aa
    bp
    rw
    """,
    figsize=(10, 12),
    layout="constrained",
)

main_ax = axs["a"]
main_ax.set_title("City of London")

for geometry_type, color in zip(geometry_types, colors):
    filename = f"files/london_{geometry_type}_example.parquet"
    gdf = gpd.read_parquet(filename, columns=["geometry"])
    for ax in (main_ax, axs[geometry_type[0]]):
        gdf.plot(ax=ax, markersize=1, zorder=1, alpha=0.8, color=color)

for geometry_type in geometry_types:
    axs[geometry_type[0]].set_title(geometry_type.capitalize())

for key, ax in axs.items():
    ax.set_xlim([bounds[0] - 0.001, bounds[2] + 0.001])
    ax.set_ylim([bounds[1] - 0.001, bounds[3] + 0.001])

    if key == "a":
        continue

    geometry_filter.plot(
        ax=ax,
        color=(0, 0, 0, 0),
        zorder=2,
        edgecolor="#EF476F",
        linewidth=1.5,
    )
    ax.set_axis_off()


## PyArrow filters

By default, `OvertureMaestro` loads all of the features intersecting given geometry.

Hovewer, there is also an option to pass filters (`--filter` or `--pyarrow-filter`) used by PyArrow during downloading step.

Filters are expected to be strings in this format: `<column name(s)> <operator> <value>`.

Examples: `confidence > 0.95`, `subtype == road`, `categories,primary = museum`.

Passed strings are parsed to [`pyarrow.Expression`](https://arrow.apache.org/docs/python/generated/pyarrow.dataset.Expression.html#pyarrow.dataset.Expression). Multiple filters can be passed to the CLI.

You can read abaout every dataset type schema and available fields [here](https://docs.overturemaps.org/schema/reference/).

Note: **Overture Maps** schema can change between release versions.

### Load only rivers

In [ ]:
! OvertureMaestro base water --filter "subtype = river" \
    --geom-filter-bbox "17.010921,51.093406,17.054266,51.122229" \
    --output files/wroclaw_rivers_example.parquet

In [ ]:
! pixel-map files/wroclaw_rivers_example.parquet --width 82 --height 23 --renderer ascii-bw

### Load railroads

In [ ]:
! OvertureMaestro transportation segment --filter "subtype = rail" \
    --geom-filter-bbox "17.010921,51.093406,17.054266,51.122229" \
    --output files/wroclaw_rail_example.parquet

In [ ]:
! ./duckdb :memory: ".mode line" "FROM 'files/wroclaw_rail_example.parquet' LIMIT 1";

### Load museums with a high confidence score

Here, the nested field `categories.primary` have to be accessed using comma delimited syntax.

In [ ]:
! OvertureMaestro places place --filter "categories,primary = museum" --filter "confidence > 0.9" \
    --geom-filter-bbox "17.010921,51.093406,17.054266,51.122229" \
    --output files/wroclaw_museums_example.parquet

In [ ]:
! ./duckdb :memory: "SELECT names['primary'], ROUND(confidence, 4) confidence FROM 'files/wroclaw_museums_example.parquet'";